In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import pygsp
import mplleaflet


March 1st: first reported case in New York 
<br>
March 4th: Yeshiva University campus closed, high school in the Bronx 
<br>
March 9th 16 confirmed cases 
<br>
March 10th: mitigation measures expanded (online classes for universities) 
<br>
March 11th: CUNY and SUNY closed
<br>
March 14th: first reported death in New York 
<br>
March 16th: NYC public schools closed 
<br>
March 20th: stay-at-home order for non-essential workers, total over 7000 cases
<br>
March 22nd: shortages of PPE for health workers, recommendation that health facilities stop testing non-hospitalized patients
<br>
March 26th: USNS Comfort heading to NYC to assist local hospitals 


New York Weather : MARCH

[weather data](https://www.timeanddate.com/weather/usa/new-york/historic?month=3&year=2020)

In [ ]:
df = pd.read_csv('./data/202003-citibike-tripdata.csv')
n_start_station = len(df['start station id'].unique())
n_end_station = len(df['end station id'].unique())


In [ ]:
total_station_id = set(df['start station id']).union(set(df['end station id']))
n_tot_station = len(total_station_id)

In [ ]:
id_index = dict(zip(sorted(total_station_id), np.arange(n_tot_station)))

In [ ]:
df.head()

# Location Parser #

In [ ]:
useful = [3, 5, 6, 7, 9, 10]
df.iloc[:,useful]

# Find Locations #

In [ ]:
locations = dict()
for e in id_index.keys():
    if df[df['start station id'] == e]['start station latitude'].shape[0]:
        locations[id_index[e]] = (df[df['start station id'] == e]['start station longitude'].iloc[0],
                                  df[df['start station id'] == e]['start station latitude'].iloc[0])
    else:
        locations[id_index[e]] = (df[df['end station id'] == e]['end station longitude'].iloc[0],
                                  df[df['end station id'] == e]['end station latitude'].iloc[0])
 

# Parser #

In [ ]:
useful = [1, 0, 3, 7, 11, 13, 14]
df.iloc[:,useful]

# Util Functions #

In [ ]:
# Find indexes to extract daily rides

def days_index():
    d_i = {}
    for index, row in df.iterrows():
        day = int(row['starttime'].split()[0].split('-')[2])
        if d_i.get(day) is None:
            d_i[day] = index
    return d_i

In [ ]:
# Dictionary with days starting indexes
"""d = days_index()
with open('./data/variables/march_index.pickle', 'wb') as file:
    pickle.dump(d, file)"""


# MARCH #

In [ ]:
# Load day indexes for March
with open('./data/variables/march_index.pickle', 'rb') as file:
    d = pickle.load(file)

In [ ]:
d_a = [e for e in d.values()]
d_a.append(df.shape[0])
d_a = np.array(d_a)
rides_day = d_a[1:] - d_a[:-1]

In [ ]:
# Rides Per Day
plt.figure(figsize=(12,8))
_ = plt.plot(rides_day)
_ = plt.xticks(np.arange(31), np.arange(31))
_ = plt.xlabel("Day of March")
_ = plt.ylabel("Rides per day")

In [ ]:
print("Maximum number of rides on day {}".format(np.argmax(rides_day)))
print("With {} rides".format(np.max(rides_day)))

## 2st-6th of March ##

In [ ]:
# 2 -> Monday
# Choose day 
days = [2, 3, 4, 5, 6]
#days = [4]

# Find all connections with weights
adj_2_6_tot = np.zeros((n_tot_station, n_tot_station))
for day in days:
    for index, row in df.iloc[d[day]:d[day+1]].iterrows():
        adj_2_6_tot[id_index[row['start station id']], id_index[row['end station id']]] += 1
        adj_2_6_tot[id_index[row['end station id']], id_index[row['start station id']]] +=1
    print('Day {} loaded...'.format(day))
    

In [ ]:
print("Total number of rides : {}".format(np.sum(adj_2_6_tot) / 2))

In [ ]:
adj_2_6 = adj_2_6_tot.copy()
adj_2_6[adj_2_6 > 0] = 1
print("Unweighted number of rides (edges) : {}".format(np.sum(adj_2_6) / 2))

for i in range(n_tot_station):
    adj_2_6[i, i] = 0

In [ ]:
# Create Undirected graph 
g_2_6 = nx.from_numpy_matrix(adj_2_6)
degs_2_6 = np.array([x[1] for x in g_2_6.degree])
_ = plt.hist(degs_2_6)

In [ ]:
plt.scatter(np.arange(n_tot_station), degs_2_6)

## Plot ##

In [ ]:
fig, ax = plt.subplots()

nx.draw_networkx_nodes(g_2_6, locations, node_size=40, node_color='gray', ax=ax)
nx.draw_networkx_edges(g_2_6, locations, alpha=0.8, width=0.2, edge_color='black', ax=ax)

mplleaflet.show()

### Clustering ###

In [ ]:
isolated_nodes_2_6 = []
for e in nx.connected_components(g_2_6):
    if len(e) < 3:
        isolated_nodes_2_6.append(e.pop())

g_2_6.remove_nodes_from(isolated_nodes_2_6)

for e in nx.connected_components(g_2_6):
    if len(e) < 3:
        print(e)

In [ ]:
G_2_6 = pygsp.graphs.Graph(nx.adjacency_matrix(g_2_6))

G_2_6.compute_laplacian()
G_2_6.compute_fourier_basis()

In [ ]:
plt.scatter(G_2_6.U[1,:], G_2_6.U[2,:])

In [ ]:
nx.write_gexf(g_2_6, "./graphs/g_2_6.gexf")

In [ ]:
nx.algorithms.cluster.average_clustering(g_2_6)

## 9th-13th of March ##

In [ ]:
# 9 -> Monday
# Choose day 
days = [9, 10, 11, 12, 13]
#days = [11]

# Find all connections with weights
adj_9_13_tot = np.zeros((n_tot_station, n_tot_station))
for day in days:
    for index, row in df.iloc[d[day]:d[day+1]].iterrows():
        adj_9_13_tot[id_index[row['start station id']], id_index[row['end station id']]] += 1
        adj_9_13_tot[id_index[row['end station id']], id_index[row['start station id']]] += 1
    print('Day {} loaded...'.format(day))
    

In [ ]:
print("Total number of rides : {}".format(np.sum(adj_9_13_tot) / 2))

In [ ]:
adj_9_13 = adj_9_13_tot.copy()
adj_9_13[adj_9_13 > 0] = 1
print("Unweighted number of rides : {}".format(np.sum(adj_9_13) / 2))

# Remove Self-loops
for i in range(n_tot_station):
    adj_9_13[i, i] = 0

In [ ]:
# Create Undirected graph 
g_9_13 = nx.from_numpy_matrix(adj_9_13)
degs_9_13 = np.array([x[1] for x in g_9_13.degree])
_ = plt.hist(degs_9_13)

In [ ]:
plt.scatter(np.arange(n_tot_station), degs_9_13)

## Plot ##

In [ ]:
fig, ax = plt.subplots()

nx.draw_networkx_nodes(g_9_13, locations, node_size=40, node_color='gray', ax=ax)
nx.draw_networkx_edges(g_9_13, locations, alpha=0.8, width=0.2, edge_color='black', ax=ax)

# mplleaflet.show()

### Clustering ###

In [ ]:
isolated_nodes_9_13 = []
for e in nx.connected_components(g_9_13):
    if len(e) < 3:
        isolated_nodes_9_13.append(e.pop())

g_9_13.remove_nodes_from(isolated_nodes_9_13)

for e in nx.connected_components(g_9_13):
    if len(e) < 3:
        print(e)

In [ ]:
G_9_13 = pygsp.graphs.Graph(nx.adjacency_matrix(g_9_13))

G_9_13.compute_laplacian()
G_9_13.compute_fourier_basis()

In [ ]:
plt.scatter(G_9_13.U[1,:], G_9_13.U[2,:])

In [ ]:
nx.algorithms.cluster.average_clustering(g_9_13)

## 16th-20th of March ##

In [ ]:
# 16 -> Monday
# Choose day 
days = [16, 17, 18, 19, 20]
#days = [18]

# Find all connections with weights
adj_16_20_tot = np.zeros((n_tot_station, n_tot_station))
for day in days:
    for index, row in df.iloc[d[day]:d[day+1]].iterrows():
        adj_16_20_tot[id_index[row['start station id']], id_index[row['end station id']]] += 1
        adj_16_20_tot[id_index[row['end station id']], id_index[row['start station id']]] += 1
    print('Day {} loaded...'.format(day))
    

In [ ]:
print("Total number of rides : {}".format(np.sum(adj_16_20_tot) / 2))

In [ ]:
adj_16_20 = adj_16_20_tot.copy()
adj_16_20[adj_16_20 > 0] = 1
print("Unweighted number of rides : {}".format(np.sum(adj_16_20) / 2))

# Remove Self-loops
for i in range(n_tot_station):
    adj_16_20[i, i] = 0

In [ ]:
# Create Undirected graph 
g_16_20 = nx.from_numpy_matrix(adj_16_20)
degs_16_20 = np.array([x[1] for x in g_16_20.degree])
_ = plt.hist(degs_16_20)

In [ ]:
plt.scatter(np.arange(n_tot_station), degs_16_20)

## Plot ## 

In [ ]:
fig, ax = plt.subplots()

nx.draw_networkx_nodes(g_16_20, locations, node_size=40, node_color='gray', ax=ax)
nx.draw_networkx_edges(g_16_20, locations, alpha=0.8, width=0.2, edge_color='black', ax=ax)

# mplleaflet.show()

### Clustering ###

In [ ]:
isolated_nodes_16_20 = []
for e in nx.connected_components(g_16_20):
    if len(e) < 3:
        isolated_nodes_16_20.append(e.pop())

g_16_20.remove_nodes_from(isolated_nodes_16_20)

for e in nx.connected_components(g_16_20):
    if len(e) < 3:
        print(e)

In [ ]:
G_16_20 = pygsp.graphs.Graph(nx.adjacency_matrix(g_16_20))

G_16_20.compute_laplacian()
G_16_20.compute_fourier_basis()

plt.scatter(G_16_20.U[1,:], G_16_20.U[2,:])

In [ ]:
nx.algorithms.cluster.average_clustering(g_16_20)

## 23st-27th of March ##

In [ ]:
# 23 -> Monday
# Choose day 
days = [23, 24, 25, 26, 27]
#days = [24]

# Find all connections with weights
adj_23_27_tot = np.zeros((n_tot_station, n_tot_station))
for day in days:
    for index, row in df.iloc[d[day]:d[day+1]].iterrows():
        adj_23_27_tot[id_index[row['start station id']], id_index[row['end station id']]] += 1
        adj_23_27_tot[id_index[row['end station id']], id_index[row['start station id']]] += 1
    print('Day {} loaded...'.format(day))
    

In [ ]:
print("Total number of rides : {}".format(np.sum(adj_23_27_tot) / 2))

In [ ]:
adj_23_27 = adj_23_27_tot.copy()
adj_23_27[adj_23_27 > 0] = 1
print("Unweighted number of rides : {}".format(np.sum(adj_23_27) / 2))

# Remove Self-loops
for i in range(n_tot_station):
    adj_23_27[i, i] = 0

In [ ]:
# Create Undirected graph 
g_23_27 = nx.from_numpy_matrix(adj_23_27)
degs_23_27 = np.array([x[1] for x in g_23_27.degree])
_ = plt.hist(degs_23_27)

In [ ]:
plt.scatter(np.arange(n_tot_station), degs_23_27)

## Plot ##

In [ ]:
fig, ax = plt.subplots()

nx.draw_networkx_nodes(g_23_27, locations, node_size=40, node_color='gray', ax=ax)
nx.draw_networkx_edges(g_23_27, locations, alpha=0.8, width=0.2, edge_color='black', ax=ax)

# mplleaflet.show()

### Clustering ###

In [ ]:
isolated_nodes_23_27 = []
for e in nx.connected_components(g_23_27):
    if len(e) < 3:
        isolated_nodes_23_27.append(e.pop())

g_23_27.remove_nodes_from(isolated_nodes_23_27)

for e in nx.connected_components(g_23_27):
    if len(e) < 3:
        print(e)

In [ ]:
G_23_27 = pygsp.graphs.Graph(nx.adjacency_matrix(g_23_27))

G_23_27.compute_laplacian()
G_23_27.compute_fourier_basis()

plt.scatter(G_23_27.U[1,:], G_23_27.U[2,:])

In [ ]:
nx.algorithms.cluster.average_clustering(g_23_27)

## Deg-Sorted Nodes over time ##

In [ ]:
sort_deg = np.argsort(degs_2_6)

In [ ]:
plt.scatter(np.arange(n_tot_station), degs_2_6[sort_deg])

In [ ]:
plt.scatter(np.arange(n_tot_station), degs_9_13[sort_deg])

In [ ]:
plt.scatter(np.arange(n_tot_station), degs_16_20[sort_deg])

In [ ]:
plt.scatter(np.arange(n_tot_station), degs_23_27[sort_deg])

# Function analysis #

## Rides Functions 2-6 ##

In [ ]:
# ATTENTION !!!
adj_2_6_tot = np.delete(adj_2_6_tot, isolated_nodes_2_6, 0)
adj_2_6_tot = np.delete(adj_2_6_tot, isolated_nodes_2_6, 1)

In [ ]:
f_2_6 = np.sum(adj_2_6_tot, axis=0)

# Compute number of access per stations

f_2_6_spect = np.dot(G_2_6.U.T, f_2_6)
f_2_6_spect /= np.sum(np.abs(f_2_6_spect))

plt.plot(np.abs(f_2_6_spect[1:]))

## Rides Functions 9-13 ##

In [ ]:
# ATTENTION !!!
adj_9_13_tot = np.delete(adj_9_13_tot, isolated_nodes_9_13, 0)
adj_9_13_tot = np.delete(adj_9_13_tot, isolated_nodes_9_13, 1)

In [ ]:
f_9_13 = np.sum(adj_9_13_tot, axis=0)
## Rides Functions 9-13 ##

# Compute number of access per stations

f_9_13_spect = np.dot(G_9_13.U.T, f_9_13)
f_9_13_spect /= np.sum(np.abs(f_9_13_spect))

plt.plot(np.abs(f_9_13_spect[1:]))

## Rides Functions 6-20 ##

In [ ]:
# ATTENTION !!!

adj_16_20_tot = np.delete(adj_16_20_tot, isolated_nodes_16_20, 0)
adj_16_20_tot = np.delete(adj_16_20_tot, isolated_nodes_16_20, 1)

In [ ]:

f_16_20 = np.sum(adj_16_20_tot, axis=0)

# Compute number of access per stations

f_16_20_spect = np.dot(G_16_20.U.T, f_16_20)
f_16_20_spect /= np.sum(np.abs(f_16_20_spect))

plt.plot(np.abs(f_16_20_spect[1:]))

## Rides Functions 23-27 ##

In [ ]:
# ATTENTION !!!

adj_23_27_tot = np.delete(adj_23_27_tot, isolated_nodes_23_27, 0)
adj_23_27_tot = np.delete(adj_23_27_tot, isolated_nodes_23_27, 1)

In [ ]:
f_23_27 = np.sum(adj_23_27_tot, axis=0)

# Compute number of access per stations

f_23_27_spect = np.dot(G_23_27.U.T, f_23_27) 
f_23_27_spect = f_23_27_spect / np.sum(np.abs(f_23_27_spect))

plt.plot(np.abs(f_23_27_spect[1:]))

In [ ]:
plt.plot(np.cumsum(np.abs(f_2_6_spect)))
plt.plot(np.cumsum(np.abs(f_9_13_spect)))

In [ ]:
plt.plot(np.cumsum(np.abs(f_9_13_spect)))
plt.plot(np.cumsum(np.abs(f_16_20_spect)))

In [ ]:
plt.plot(np.cumsum(np.abs(f_2_6_spect)))
plt.plot(np.cumsum(np.abs(f_23_27_spect)))

# Functions on Graphs #

In [ ]:
d_2_6 = np.delete(degs_2_6, isolated_nodes_2_6)
d_9_13 = np.delete(degs_9_13, isolated_nodes_9_13)
d_16_20 = np.delete(degs_16_20, isolated_nodes_16_20)
d_23_27 = np.delete(degs_23_27, isolated_nodes_23_27)

In [ ]:
plt.figure(figsize=(12,16))
plt.subplot(2,2,1)
nx.draw_networkx_nodes(g_2_6, locations, node_size=40, node_color=f_2_6)
nx.draw_networkx_edges(g_2_6, locations, alpha=0.8, width=0.2, edge_color='black')

plt.subplot(2,2,2)
nx.draw_networkx_nodes(g_9_13, locations, node_size=40, node_color=f_9_13)
nx.draw_networkx_edges(g_9_13, locations, alpha=0.8, width=0.2, edge_color='black')

plt.subplot(2,2,3)
nx.draw_networkx_nodes(g_16_20, locations, node_size=40, node_color=f_16_20)
nx.draw_networkx_edges(g_16_20, locations, alpha=0.8, width=0.2, edge_color='black')

plt.subplot(2,2,4)
nx.draw_networkx_nodes(g_23_27, locations, node_size=40, node_color=f_23_27)
nx.draw_networkx_edges(g_23_27, locations, alpha=0.8, width=0.2, edge_color='black')



In [ ]:
plt.figure(figsize=(12,16))
plt.subplot(2,2,1)

nx.draw_networkx_nodes(g_2_6, locations, node_size=40, node_color=d_2_6)
nx.draw_networkx_edges(g_2_6, locations, alpha=0.8, width=0.2, edge_color='black')

plt.subplot(2,2,2)
nx.draw_networkx_nodes(g_9_13, locations, node_size=40, node_color=d_9_13)
nx.draw_networkx_edges(g_9_13, locations, alpha=0.8, width=0.2, edge_color='black')

plt.subplot(2,2,3)
nx.draw_networkx_nodes(g_16_20, locations, node_size=40, node_color=d_16_20)
nx.draw_networkx_edges(g_16_20, locations, alpha=0.8, width=0.2, edge_color='black')

plt.subplot(2,2,4)
nx.draw_networkx_nodes(g_23_27, locations, node_size=40, node_color=d_23_27)
nx.draw_networkx_edges(g_23_27, locations, alpha=0.8, width=0.2, edge_color='black')

In [ ]:
plt.plot(G_2_6.e)
plt.plot(G_9_13.e)
plt.plot(G_16_20.e)
plt.plot(G_23_27.e)

In [ ]:
plt.plot(G_2_6.e[:50])
plt.plot(G_9_13.e[:50])
plt.plot(G_16_20.e[:50])
plt.plot(G_23_27.e[:50])

In [ ]:
G_2_6.e[:10]

In [ ]:
G_9_13.e[:10]

In [ ]:
G_16_20.e[:10]

In [ ]:
G_23_27.e[:10]